In [ ]:
import os
import statsmodels.api as sm
RAND_SEED = 12345
import numpy as np
np.random.seed(RAND_SEED)
import random
random.seed(RAND_SEED)
from statsmodels.tools.tools import add_constant
import pandas as pd
import statsmodels

In [ ]:
# Define directories and file names
cwd = os.getcwd()
DataFolder = os.path.join(cwd, 'data')
InputDataFile = os.path.join(DataFolder, 'Critical_Oil_Rate.csv')

In [ ]:

import gdown

url = ' https://drive.google.com/uc?id=1UQPnvJWEZFfImaKB9TIVPSWZJabSdhXH'
InputDataFile = 'Critical_Oil_Rate.csv'


gdown.download(url, InputDataFile, quiet=False)

Downloading...
From:  https://drive.google.com/uc?id=1UQPnvJWEZFfImaKB9TIVPSWZJabSdhXH
To: /content/Critical_Oil_Rate.csv
100%|██████████| 1.85k/1.85k [00:00<00:00, 5.37MB/s]


'Critical_Oil_Rate.csv'

In [ ]:
# read Input  file and show headers
df = pd.read_csv(InputDataFile)
df.head()

,Fracture effective permeability,Matrix permeability,Mobility,Fracture spacing,drainage radius,Anisotropy ratio,Penetration ratio,Critical Oil Rate
0,500,5.0,3,0.1,400,1,0.5,22.7
1,500,5.0,3,0.1,400,10,0.5,19.4
2,500,5.0,3,0.1,2000,1,0.5,18.9
3,500,5.0,3,0.1,2000,10,0.5,18.2
4,500,5.0,3,18.0,400,1,0.5,22.7


In [ ]:
# describe input file with key statistics values
df.describe()

,Fracture effective permeability,Matrix permeability,Mobility,Fracture spacing,drainage radius,Anisotropy ratio,Penetration ratio,Critical Oil Rate
count,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000
mean,712.903226,12.761290,2.967742,7.180645,1077.419355,5.193548,0.529032,33.854839
std,654.238105,18.489424,1.279956,5.810246,511.320808,2.833284,0.145858,43.424650
min,100.000000,0.100000,1.000000,0.100000,400.000000,1.000000,0.350000,1.800000
25%,500.000000,5.000000,3.000000,6.000000,1000.000000,5.000000,0.500000,10.275000
50%,500.000000,5.000000,3.000000,6.000000,1000.000000,5.000000,0.500000,19.000000
75%,500.000000,5.000000,3.000000,6.000000,1000.000000,5.000000,0.500000,47.000000
max,2000.000000,50.000000,5.000000,18.000000,2000.000000,10.000000,0.800000,240.000000


In [ ]:
# Define variable to be used later in Aspect_ratio and Normalized_criticalrate
Kf = df.iloc[:,0].values  # Fracture effective permeability
Km = df.iloc[:,1].values  # Matrix permeability
M  = df.iloc[:,2].values  # Mobility
Sp = df.iloc[:,3].values  # Fracture spacing
re = df.iloc[:,4].values  # Drainage radius
I  = df.iloc[:,5].values  # Anisotropy ratio
p  = df.iloc[:,6].values  # Penetration ratio
Cr = df.iloc[:,7].values  # Critical Oil Rate

In [ ]:
# Compute Aspect_ratio and Normalized_criticalrate
A  = re/40*(I)**0.5   # Aspect_ratio
Y  = Cr/(0.0783*(10**-4)*(Kf+Km*0.63)*40**2*(1-p**2)*(64-52)/(0.7*M)/1.1)  # Normalized_criticalrate

In [ ]:
# Add A and Y to the dataframe
df['Aspect_ratio'] = A
df['Normalized_criticalrate'] = Y
df.head()

,Fracture effective permeability,Matrix permeability,Mobility,Fracture spacing,drainage radius,Anisotropy ratio,Penetration ratio,Critical Oil Rate,Aspect_ratio,Normalized_criticalrate
0,500,5.0,3,0.1,400,1,0.5,22.7,10.000000,0.924307
1,500,5.0,3,0.1,400,10,0.5,19.4,31.622777,0.789936
2,500,5.0,3,0.1,2000,1,0.5,18.9,50.000000,0.769577
3,500,5.0,3,0.1,2000,10,0.5,18.2,158.113883,0.741074
4,500,5.0,3,18.0,400,1,0.5,22.7,10.000000,0.924307


In [ ]:
# Compute input feature X = 1/A + constant
x = 1/A
X = sm.add_constant(x) # adding a constant

In [ ]:
# Build OLS Regression model and fit input data
model = sm.OLS(Y, X).fit()

In [ ]:
# Make predictions
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     339.5
Date:                Sat, 12 Jul 2025   Prob (F-statistic):           2.21e-26
Time:                        08:00:19   Log-Likelihood:                 170.01
No. Observations:                  62   AIC:                            -336.0
Df Residuals:                      60   BIC:                            -331.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7277      0.003    214.698      0.000       0.721       0.734
x1             1.9969      0.108     18.426      0.000       1.780       2.214
==============================================================================
Omnibus:                      119.548   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5122.268
Skew:                          -6.193   Prob(JB):                         0.00
Kurtosis:                      45.772   Cond. No.                         53.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""